**IP.2** [Facility location]
A company is considering opening warehouses in four cities: New York, Los Angeles, Chicago and Atlanta. Each warehouse can ship 100 units per week. The weekly fixed cost of keeping each warehouse open is \\$400 for New York, \\$500 for Los Angeles, \\$300 for Chicago and \\$150 for Atlanta. Region 1 of the country requires 80 units per week, region 2 requires 70 units per week and region 3 requires 40 units per week. The costs (including production and shipping costs) of sending one unit from a plant
to a region are shown in the table below.

  
| From/To       | Region 1      | Region 2|Region 3|
| ------------- |:-------------:| -------:|-------:|
| NY      | 20 | 40 |50|
| LA      | 48      |   15 |26|
| Chicago| 26     |    35 |18|
|    Atlanta           |        24       |      50   |35|

****Part a)**** Formulate an IP to meet weekly demands at minimum costs.

First, create the model object and the decision variables.

In [395]:
import gurobipy as gp
from gurobipy import GRB,quicksum

#Create a model object
m=gp.Model('Facility Location')

In [396]:
cities=['N','L','C','A']
regions=[1,2,3]

#Uncomment and fill the code below to create the decision variables
x=m.addVars(cities,regions,lb= 0 ,vtype= GRB.INTEGER )
y=m.addVars(cities,vtype= GRB.BINARY ) 

Next, we specify the objective function:

In [397]:
#Lists corresponding to costs
shipping_costs={'N':[20,40,50],'L':[48,15,26],'C':[26,35,18],'A':[24,50,35]}
fixed_cost={'N':400,'L':500,'C':300,'A':150}

#Uncomment and fill the code below to specify the objective function
m.setObjective(
    quicksum(fixed_cost[c] * y[c] for c in cities) + 
    quicksum(shipping_costs[c][r-1] * x[c,r] * y[c] for c in cities for r in regions), 
    GRB.MINIMIZE
)

Add constraints ensuring that the demand for each region is satisfied:

In [398]:
#Demand for each region
demands={1:80,2:70,3:40}

#Insert your code here:
m.addConstrs(
    quicksum(x[c,r] for c in cities) >= demands[r] for r in regions
)

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

Add constraints ensuring that each warehouse can ship at most 100 units per week:

In [399]:
#Insert your code here:
m.addConstrs(
    quicksum(x[c,r] for r in regions) <= 100*y[c] for c in cities
)

{'N': <gurobi.Constr *Awaiting Model Update*>,
 'L': <gurobi.Constr *Awaiting Model Update*>,
 'C': <gurobi.Constr *Awaiting Model Update*>,
 'A': <gurobi.Constr *Awaiting Model Update*>}

****Part b)**** Write a constraint for: if the New York warehouse is opened, then the Los Angeles warehouse must be opened.

Find an inequality involving the binary variable for New York and that for Los Angeles. 

In [400]:
#Insert your code here:
m.addConstr(
    y['L'] >= y['N']
)

<gurobi.Constr *Awaiting Model Update*>

****Part c)**** Write a constraint for:  at most two warehouses can be opened.

In [401]:
#Insert your code here:
m.addConstr(
    quicksum(y[c] for c in cities) <= 2
)

<gurobi.Constr *Awaiting Model Update*>

****Part d)**** Write a constraint for:  either the Atlanta or the Los Angeles warehouse must be opened.

In [402]:
#Insert your code here:
m.addConstr(
    y['A'] + y['L'] <= 1
)
m.addConstr(
    y['A'] + y['L'] >=1
)

<gurobi.Constr *Awaiting Model Update*>

****Part e)**** Write a constraint for:  for shipping to be economically feasible, at least 20 units must be shipped.

In [403]:
#Insert your code here:
# m.addConstrs(
#     x[c,r]**2  >= 20 * x[c,r] for c in cities for r in regions
# )
# this is not a linear constraint, 
# since there is no linear constraint without an auxiliary variable that will
# accurately reflect this constraint

# we could do the same thing with auxiliary variables, 
# which would require an auxiliary variable for each combination of region and city
aux = m.addVars(cities, regions, vtype = GRB.BINARY)
M = 99999
m.addConstrs( x[c,r] >= 20 - M * aux[c,r] for c in cities for r in regions)
m.addConstrs( x[c,r] <= 0 + M * (1-aux[c,r]) for c in cities for r in regions)

# alternatively, we could require each operating warehouse to ship at least 
# 20 units to each region. However, this would be a binding constraint and change
# our optimal solution compared to the other two options.
# The constraint would then be:
# m.addConstrs( x[c,r] >= 20*y[c] for c in cities for r in regions)

m.update()

****Part f)**** Write constraints for:  if New York ships to region 1, then no other warehouses can ship to region 1.

First, introduce a new binary variable that will indicate if New York ships to region 1.

In [404]:
#Uncomment and fill the code below to add the new binary variable:

y_N1 = m.addVar(lb = 0, vtype = GRB.BINARY)

Now, introduce constraints that ensure $y_{N1}$(the new variable) is 1 when $x_{N1}$ is positive, and 0 otherwise.

In [405]:
#Uncomment and fill the code below:
M= 9999999
m.addConstr(x['N',1] <= 0 + M*y_N1)

<gurobi.Constr *Awaiting Model Update*>

Next, using the new variable, add a new constraint to make sure no other warehouse can ship to region 1 if New York ships to region 1.

In [406]:
#Insert your code here:
m.addConstr(x['L',1] + x['C',1] + x['A',1] <= 0 + M * (1- y_N1 ))

<gurobi.Constr *Awaiting Model Update*>

Finally, we solve the model and printout the optimal revenue!

In [407]:
def printSolution():
    if m.status == GRB.OPTIMAL:
        print('\nTotal costs: %g' % m.objVal)
        print('\nShipped amounts:')
        shipped_amountsx = m.getAttr('x', x)
        for c in cities:
            for r in regions:            
                print('%s to %s: %g' % (c,r, shipped_amountsx[c,r]))
    else:
        print('No solution:', m.status)

m.params.NonConvex = 2
m.optimize()
printSolution()

Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 37 rows, 29 columns and 92 nonzeros
Model fingerprint: 0xdf43e1c8
Model has 12 quadratic objective terms
Variable types: 0 continuous, 29 integer (17 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [2e+02, 5e+02]
  QObjective range [3e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Found heuristic solution: objective 8150.0000000
Presolve removed 3 rows and 2 columns
Presolve time: 0.01s
Presolved: 46 rows, 39 columns, 120 nonzeros
Variable types: 0 continuous, 39 integer (15 binary)
Found heuristic solution: objective 8132.0000000

Root relaxation: objective 5.258333e+02, 24 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl